In [1]:
function only(itr)
    state = start(itr)
    val,state = next(itr, state)
    @assert(done(itr,state))
    return val
end

only (generic function with 1 method)

In [2]:
using SymEngine
import SymEngine: as_numer_denom
using Expr2LaTeX
Expr2LaTeX.render(symexpr::SymEngine.Basic)=symexpr |> repr |> parse |> render

In [3]:
function ugly_simplify(f)
    num,den = as_numer_denom(f)
    inv(expand(inv(num)*den))
end

function basic_simplify(f)
    num2,den2 = as_numer_denom(ugly_simplify(f))
    num2/den2
end

(a)≖(b) = ugly_simplify(a)==ugly_simplify(b)

### Tests
using Base.Test
@vars z
s1 = (1 + exp(-z))^(-1)
s2 = exp(z)/(1+exp(z))

@test s1 ≖ s2
@test inv(s1)≖inv(s2)

n1 = subs(s1, z=>-z)
n2 = 1-s1
n3 = subs(s2, z=>-z)
n4 = 1-s2
@test n1 ≖ n2 ≖ n3 ≖ n4
@test inv(n1)≖inv(n2) ≖ inv(n3) ≖ inv(n4)


Test Passed


In [4]:
sigmoid(z)=(1 + exp(-z))^(-1)


sigmoid (generic function with 1 method)

In [5]:
sigmoid(z)=(1 + exp(-z))^(-1)
@vars z
diff(sigmoid(z),z)

exp(-z)/(1 + exp(-z))^2

In [6]:
function matrix_var(name, input_size, output_size)
    [symbols("$(name)_$j$i") for j in 1:output_size, i in 1:input_size]
end
function vector_var(name, output_size)
    [symbols("$(name)_$j") for j in 1:output_size]
end

const weight_var = matrix_var
const bias_var = vector_var


vector_var (generic function with 1 method)

## Basic Network

Single input single output
1 hidden layer

In [7]:
@vars x
W = weight_var("W", 1, 3)
b = bias_var("b",3)
V = weight_var("V", 3, 1)

z=(sigmoid.(W*x + b))
N=V*z |> only
render(N)

L"$(\frac{V_{11}}{(1 + \mathrm{exp}(-(b_{1} + (x * W_{11}))))} + \frac{V_{12}}{(1 + \mathrm{exp}(-(b_{2} + (x * W_{21}))))} + \frac{V_{13}}{(1 + \mathrm{exp}(-(b_{3} + (x * W_{31}))))})$"

In [8]:
dNdx = diff(N,x)
render(dNdx)

L"$(\frac{(\mathrm{exp}(-(b_{1} + (x * W_{11}))) * W_{11} * V_{11})}{(1 + \mathrm{exp}(-(b_{1} + (x * W_{11}))))^{2}} + \frac{(\mathrm{exp}(-(b_{2} + (x * W_{21}))) * W_{21} * V_{12})}{(1 + \mathrm{exp}(-(b_{2} + (x * W_{21}))))^{2}} + \frac{(\mathrm{exp}(-(b_{3} + (x * W_{31}))) * W_{31} * V_{13})}{(1 + \mathrm{exp}(-(b_{3} + (x * W_{31}))))^{2}})$"

In [9]:
# I am pretty sure this is the right equation for the derivative of a plain nerual network
# But symengine is not strong enough to  recognise that it matches the above
mdNdx = (W'.*V)*(sigmoid.(-(W*x + b)).*sigmoid.(W*x + b)) |> first
render.(mdNdx)

L"$(\frac{(W_{11} * V_{11})}{((1 + \mathrm{exp}(b_{1} + (x * W_{11}))) * (1 + \mathrm{exp}(-(b_{1} + (x * W_{11})))))} + \frac{(W_{21} * V_{12})}{((1 + \mathrm{exp}(b_{2} + (x * W_{21}))) * (1 + \mathrm{exp}(-(b_{2} + (x * W_{21})))))} + \frac{(W_{31} * V_{13})}{((1 + \mathrm{exp}(b_{3} + (x * W_{31}))) * (1 + \mathrm{exp}(-(b_{3} + (x * W_{31})))))})$"

In [10]:
W'.*V

1×3 Array{SymEngine.Basic,2}:
 W_11*V_11  W_21*V_12  W_31*V_13

## Expodential Output net
Single input single output still,
1 hidden layer,
output expodential,

In [11]:
@vars x
W = weight_var("W", 1, 3)
b = bias_var("b",3)
V = weight_var("V", 3, 1)

z=sigmoid.(W*x + b)
N=exp.(V*z) |> only
render(N)

L"$\mathrm{exp}(\frac{V_{11}}{(1 + \mathrm{exp}(-(b_{1} + (x * W_{11}))))} + \frac{V_{12}}{(1 + \mathrm{exp}(-(b_{2} + (x * W_{21}))))} + \frac{V_{13}}{(1 + \mathrm{exp}(-(b_{3} + (x * W_{31}))))})$"

In [12]:
dNdx = diff(N,x)
render(dNdx)

L"$(\mathrm{exp}(\frac{V_{11}}{(1 + \mathrm{exp}(-(b_{1} + (x * W_{11}))))} + \frac{V_{12}}{(1 + \mathrm{exp}(-(b_{2} + (x * W_{21}))))} + \frac{V_{13}}{(1 + \mathrm{exp}(-(b_{3} + (x * W_{31}))))}) * (\frac{(\mathrm{exp}(-(b_{1} + (x * W_{11}))) * W_{11} * V_{11})}{(1 + \mathrm{exp}(-(b_{1} + (x * W_{11}))))^{2}} + \frac{(\mathrm{exp}(-(b_{2} + (x * W_{21}))) * W_{21} * V_{12})}{(1 + \mathrm{exp}(-(b_{2} + (x * W_{21}))))^{2}} + \frac{(\mathrm{exp}(-(b_{3} + (x * W_{31}))) * W_{31} * V_{13})}{(1 + \mathrm{exp}(-(b_{3} + (x * W_{31}))))^{2}}))$"

In [13]:
# I am pretty sure this is the right equation for the derivative of this network
# as we just use the standard `d(exp(f(x))/dx = f'(x)*exp(f(x))` rule
$
# But symengine is not strong enough to  recognise that it matches the above
mdZm1dx = (W'.*V)*(sigmoid.(-(W*x + b)).*sigmoid.(W*x + b)) |> first
mdNdx = mdZm1dx*N
render.(mdNdx)

L"$(\mathrm{exp}(\frac{V_{11}}{(1 + \mathrm{exp}(-(b_{1} + (x * W_{11}))))} + \frac{V_{12}}{(1 + \mathrm{exp}(-(b_{2} + (x * W_{21}))))} + \frac{V_{13}}{(1 + \mathrm{exp}(-(b_{3} + (x * W_{31}))))}) * (\frac{(W_{11} * V_{11})}{((1 + \mathrm{exp}(b_{1} + (x * W_{11}))) * (1 + \mathrm{exp}(-(b_{1} + (x * W_{11})))))} + \frac{(W_{21} * V_{12})}{((1 + \mathrm{exp}(b_{2} + (x * W_{21}))) * (1 + \mathrm{exp}(-(b_{2} + (x * W_{21})))))} + \frac{(W_{31} * V_{13})}{((1 + \mathrm{exp}(b_{3} + (x * W_{31}))) * (1 + \mathrm{exp}(-(b_{3} + (x * W_{31})))))}))$"

## Deeper Expodential Output net
Single input single output still,
2 hidden layer,
output expodential,

In [14]:
@vars x
W = weight_var("W", 1, 3)
b = bias_var("b",3)
U = weight_var("U", 3, 3)
c = bias_var("c",3)
V = weight_var("V", 3, 1)

z1=sigmoid.(W*x + b)
z2=sigmoid.(U*z1 + c)
N=exp.(V*z2) |> only
render(N)

L"$\mathrm{exp}(\frac{V_{11}}{(1 + \mathrm{exp}(-(c_{1} + \frac{U_{11}}{(1 + \mathrm{exp}(-(b_{1} + (x * W_{11}))))} + \frac{U_{12}}{(1 + \mathrm{exp}(-(b_{2} + (x * W_{21}))))} + \frac{U_{13}}{(1 + \mathrm{exp}(-(b_{3} + (x * W_{31}))))})))} + \frac{V_{12}}{(1 + \mathrm{exp}(-(c_{2} + \frac{U_{21}}{(1 + \mathrm{exp}(-(b_{1} + (x * W_{11}))))} + \frac{U_{22}}{(1 + \mathrm{exp}(-(b_{2} + (x * W_{21}))))} + \frac{U_{23}}{(1 + \mathrm{exp}(-(b_{3} + (x * W_{31}))))})))} + \frac{V_{13}}{(1 + \mathrm{exp}(-(c_{3} + \frac{U_{31}}{(1 + \mathrm{exp}(-(b_{1} + (x * W_{11}))))} + \frac{U_{32}}{(1 + \mathrm{exp}(-(b_{2} + (x * W_{21}))))} + \frac{U_{33}}{(1 + \mathrm{exp}(-(b_{3} + (x * W_{31}))))})))})$"

In [15]:
dNdx = diff(N,x)
render(dNdx)

L"$(\mathrm{exp}(\frac{V_{11}}{(1 + \mathrm{exp}(-(c_{1} + \frac{U_{11}}{(1 + \mathrm{exp}(-(b_{1} + (x * W_{11}))))} + \frac{U_{12}}{(1 + \mathrm{exp}(-(b_{2} + (x * W_{21}))))} + \frac{U_{13}}{(1 + \mathrm{exp}(-(b_{3} + (x * W_{31}))))})))} + \frac{V_{12}}{(1 + \mathrm{exp}(-(c_{2} + \frac{U_{21}}{(1 + \mathrm{exp}(-(b_{1} + (x * W_{11}))))} + \frac{U_{22}}{(1 + \mathrm{exp}(-(b_{2} + (x * W_{21}))))} + \frac{U_{23}}{(1 + \mathrm{exp}(-(b_{3} + (x * W_{31}))))})))} + \frac{V_{13}}{(1 + \mathrm{exp}(-(c_{3} + \frac{U_{31}}{(1 + \mathrm{exp}(-(b_{1} + (x * W_{11}))))} + \frac{U_{32}}{(1 + \mathrm{exp}(-(b_{2} + (x * W_{21}))))} + \frac{U_{33}}{(1 + \mathrm{exp}(-(b_{3} + (x * W_{31}))))})))}) * (\frac{(\mathrm{exp}(-(c_{1} + \frac{U_{11}}{(1 + \mathrm{exp}(-(b_{1} + (x * W_{11}))))} + \frac{U_{12}}{(1 + \mathrm{exp}(-(b_{2} + (x * W_{21}))))} + \frac{U_{13}}{(1 + \mathrm{exp}(-(b_{3} + (x * W_{31}))))})) * V_{11} * (\frac{(\mathrm{exp}(-(b_{1} + (x * W_{11}))) * W_{11} * U_{11})}{(1 +

In [16]:
# TODO workout the clean form of this

### Find the parts that count to the gradient

In [17]:
using MacroTools: postwalk
tree = parse(repr(dNdx))

tree = postwalk(x -> @capture(x, exp(z_)) ? 1 : x, tree)
tree = postwalk(x -> @capture(x, (1+1)^2) ? 1 : x, tree)
tree = postwalk(x -> @capture(x, z_/1) ? z : x, tree)
tree = postwalk(x -> @capture(x, 1*z_) ? z : x, tree)
tree = postwalk(x -> @capture(x, 1*z_*y_) ? :($z*$y) : x, tree)
tree |> render

LoadError: [91mUndefVarError: @capture not defined[39m

In [18]:
V*(U*W )|> only

V_11*(W_11*U_11 + W_21*U_12 + W_31*U_13) + V_12*(W_11*U_21 + W_21*U_22 + W_31*U_23) + V_13*(W_11*U_31 + W_21*U_32 + W_31*U_33)

Note that each product term is scaled with a sigmoidal term that varies both with the value of the weights in that term and te corresponding biases, as well as with $x$

## Multi-output Multi-input network

matrix equations actually come out all the same as before


In [26]:
x = vector_var("x", 2)
W = matrix_var("W", 2, 3)
b = vector_var("b",3)
V = matrix_var("V", 3, 2)

z=(sigmoid.(W*x + b))
N=V*z
render.(N)

L"$(\frac{V_{11}}{(1 + \mathrm{exp}(-(b_{1} + (x_{1} * W_{11}) + (x_{2} * W_{12}))))} + \frac{V_{12}}{(1 + \mathrm{exp}(-(b_{2} + (x_{1} * W_{21}) + (x_{2} * W_{22}))))} + \frac{V_{13}}{(1 + \mathrm{exp}(-(b_{3} + (x_{1} * W_{31}) + (x_{2} * W_{32}))))})$"

L"$(\frac{V_{21}}{(1 + \mathrm{exp}(-(b_{1} + (x_{1} * W_{11}) + (x_{2} * W_{12}))))} + \frac{V_{22}}{(1 + \mathrm{exp}(-(b_{2} + (x_{1} * W_{21}) + (x_{2} * W_{22}))))} + \frac{V_{23}}{(1 + \mathrm{exp}(-(b_{3} + (x_{1} * W_{31}) + (x_{2} * W_{32}))))})$"

2-element Array{Void,1}:
 nothing
 nothing

In [29]:
dNdx = diff.(N,x)
render.(dNdx)

L"$(\frac{(\mathrm{exp}(-(b_{1} + (x_{1} * W_{11}) + (x_{2} * W_{12}))) * W_{11} * V_{11})}{(1 + \mathrm{exp}(-(b_{1} + (x_{1} * W_{11}) + (x_{2} * W_{12}))))^{2}} + \frac{(\mathrm{exp}(-(b_{2} + (x_{1} * W_{21}) + (x_{2} * W_{22}))) * W_{21} * V_{12})}{(1 + \mathrm{exp}(-(b_{2} + (x_{1} * W_{21}) + (x_{2} * W_{22}))))^{2}} + \frac{(\mathrm{exp}(-(b_{3} + (x_{1} * W_{31}) + (x_{2} * W_{32}))) * W_{31} * V_{13})}{(1 + \mathrm{exp}(-(b_{3} + (x_{1} * W_{31}) + (x_{2} * W_{32}))))^{2}})$"

L"$(\frac{(\mathrm{exp}(-(b_{1} + (x_{1} * W_{11}) + (x_{2} * W_{12}))) * W_{12} * V_{21})}{(1 + \mathrm{exp}(-(b_{1} + (x_{1} * W_{11}) + (x_{2} * W_{12}))))^{2}} + \frac{(\mathrm{exp}(-(b_{2} + (x_{1} * W_{21}) + (x_{2} * W_{22}))) * W_{22} * V_{22})}{(1 + \mathrm{exp}(-(b_{2} + (x_{1} * W_{21}) + (x_{2} * W_{22}))))^{2}} + \frac{(\mathrm{exp}(-(b_{3} + (x_{1} * W_{31}) + (x_{2} * W_{32}))) * W_{32} * V_{23})}{(1 + \mathrm{exp}(-(b_{3} + (x_{1} * W_{31}) + (x_{2} * W_{32}))))^{2}})$"

2-element Array{Void,1}:
 nothing
 nothing

In [32]:
mdZm1dx = (W'.*V)*(sigmoid.(-(W*x + b)).*sigmoid.(W*x + b)) 
render.(mdZm1dx)

L"$(\frac{(W_{11} * V_{11})}{((1 + \mathrm{exp}(b_{1} + (x_{1} * W_{11}) + (x_{2} * W_{12}))) * (1 + \mathrm{exp}(-(b_{1} + (x_{1} * W_{11}) + (x_{2} * W_{12})))))} + \frac{(W_{21} * V_{12})}{((1 + \mathrm{exp}(b_{2} + (x_{1} * W_{21}) + (x_{2} * W_{22}))) * (1 + \mathrm{exp}(-(b_{2} + (x_{1} * W_{21}) + (x_{2} * W_{22})))))} + \frac{(W_{31} * V_{13})}{((1 + \mathrm{exp}(b_{3} + (x_{1} * W_{31}) + (x_{2} * W_{32}))) * (1 + \mathrm{exp}(-(b_{3} + (x_{1} * W_{31}) + (x_{2} * W_{32})))))})$"

L"$(\frac{(W_{12} * V_{21})}{((1 + \mathrm{exp}(b_{1} + (x_{1} * W_{11}) + (x_{2} * W_{12}))) * (1 + \mathrm{exp}(-(b_{1} + (x_{1} * W_{11}) + (x_{2} * W_{12})))))} + \frac{(W_{22} * V_{22})}{((1 + \mathrm{exp}(b_{2} + (x_{1} * W_{21}) + (x_{2} * W_{22}))) * (1 + \mathrm{exp}(-(b_{2} + (x_{1} * W_{21}) + (x_{2} * W_{22})))))} + \frac{(W_{32} * V_{23})}{((1 + \mathrm{exp}(b_{3} + (x_{1} * W_{31}) + (x_{2} * W_{32}))) * (1 + \mathrm{exp}(-(b_{3} + (x_{1} * W_{31}) + (x_{2} * W_{32})))))})$"

2-element Array{Void,1}:
 nothing
 nothing

# Proof of Ability to Approximate any PDF


In [20]:
@vars z
@show diff(sigmoid(z),z) ≖ sigmoid(z)*sigmoid(-z)
sigmoid(z)*sigmoid(-z) |> render
(sigmoid(z)*sigmoid(-z)) |> ugly_simplify |> render

L"$\frac{1}{((1 + \mathrm{exp}(z)) * (1 + \mathrm{exp}(-(z))))}$"

L"$(2 + \mathrm{exp}(z) + \mathrm{exp}(-(z)))^{-1}$"

diff(sigmoid(z), z) ≖ sigmoid(z) * sigmoid(-z) = true


In [21]:
basic_simplify((sigmoid(z)*sigmoid(-z)) - exp(-z^2)) |> render

L"$\frac{(((-1 - (2 * \mathrm{exp}(z))) - \mathrm{exp}(2 * z)) + \mathrm{exp}(z + z^{2}))}{((2 * \mathrm{exp}(z + z^{2})) + \mathrm{exp}((2 * z) + z^{2}) + \mathrm{exp}(z^{2}))}$"

Our approximation to a PDF is:
$$f(x)=\frac{h(x)}{\int_{S}h(z)dz}=\frac{\frac{\partial N(x,\tilde{p})}{\partial x}}{N(max(S),\tilde{p})-N(min(S),\tilde{p})}$$.

As we have shown above the numerator $\frac{\partial N(x,\tilde{p})}{\partial x} = g(x,\tilde{p}) \;N(x,\tilde{p})$.

Where $g(x,\tilde{p})$ is  effectively a weighted sum of terms of forms similar to $\frac{\partial \sigma(z)}{\partial z} = \sigma(z)\sigma(-z) = \frac{1}{(1+\exp(-z)(1 + \exp(-z))}$.
As shown below that function defines a bell-curve.

The weights and baises of the penultimate layers of the network ($W$, $U$), define the shape and position of that bell-curve.
The  weight component of final expodential layer ($V$), determines the weighting of each bell-curve (though it also must scaled to counter effects of the prior layer weights).


$$f(x)=\frac{h(x)}{\int_{S}h(z)dz} = \frac{g(x,\tilde{p}) \; N(x,\tilde{p})}{N(max(S),\tilde{p})-N(min(S),\tilde{p})}$$.
For now we will focus on $g(x,\tilde{p})$


In [22]:
using Plots
plot(x->sigmoid(x)*sigmoid(-x))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 38.8117,375.813 41.4859,375.298 66.2124,369.185 90.9389,359.788 105.798,351.921 120.658,341.869 133.222,331.292 145.785,318.457 159.433,301.562 173.08,281.199 
 185.662,259.098 198.244,233.724 211.733,203.084 225.222,169.533 240.851,128.676 256.479,88.5685 263.424,71.9973 270.368,56.7333 277.312,43.1626 284.257,31.653 
 290.346,23.5184 296.436,17.4114 302.525,13.4771 308.615,11.811 315.254,12.6255 321.893,16.1599 328.533,22.3119 335.172,30.9069 341.851,41.7788 348.529,54.5934 
 355.208,69.0265 361.887,84.7361 376.655,122.309 391.424,161.043 405.475,196.513 419.527,229.043 426.832,244.489 434.138,258.848 441.443,272.098 448.748,284.248 
 461.047,302.304 473.346,317.554 488.362,332.803 503.377,344.891 517.493,353.874 531.608,360.979 554.221,369.337 576.835,374.976 580.315,375.661 
 "/>
 
 
 
 
 y1

##### It has been shown in [Bacharoglou, 2010, APPROXIMATION OF PROBABILITY DISTRIBUTIONS
BY CONVEX MIXTURES OF GAUSSIAN MEASURES](http://www.mathaware.org/journals/proc/2010-138-07/S0002-9939-10-10340-2/S0002-9939-10-10340-2.pdf)
that every PDF with compact support can be approximated by an weighted non-negative sum of gaussians. 
(It actually does it with a restricted subset of gaussians, but we don't need that)


Bacharogluo,2010 theorem 1 is noted as applicable to mixtures of any approximate identity ([defn](http://people.math.gatech.edu/~heil/7338/fall09/approxid.pdf)).   
It is easy to construct an approximate identity from sequences of $\frac{\partial \sigma(Wz+b)}{\partial z}$,
(though that depends on which form or approximate identity is there are many definitions)

Which directly gives us the result that $g(x, \tilde{p})$ can approximate any PDF with continuous support.
